# simple implementation of SMO

In [15]:
# import relative libraries
from numpy import *
import numpy as np

## 1. Define auxiliary functions

In [2]:
# Load data set from text files
def loadDataSet(filename):
    dataMat = []
    labelMat = []
    f = open(filename)
    for line in f.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

In [3]:
# get a random j belonging to uniform[0, m) && j != i
def selectJrand(i, m):
    j = i
    while(j == i):
        j = int(random.uniform(0,m))
    return j

In [4]:
# limit alpha between [L, H]
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
    return aj

## 2. simplified SMO

Define
$$
f(x;i) = (\alpha \cdot y)^{\rm{T}}xx_i^{\rm{T}}+b
$$
Here:
- y -- label
- x -- feature matrix
- $\alpha$ -- Lagrangian multiplier
- b -- bias


In [48]:
# see formula above↑
def fX(alphas, labelMat, dataMatrix, b, i):
    return float(multiply(alphas, labelMat).T \
                 * (dataMatrix*dataMatrix[i,:].T)) + b

In [ ]:
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMatrix = mat(dataMatIn)
    labelMat = mat(classLabels).transpose()
    b = 0
    [m,n] = shape(dataMatrix)
    alphas = mat(zeros(m, 1))
    iter = 0
    while iter < maxIter:
        alphaPairsChanged = 0
        for i in range(m):
            fXi = fX(alphas, labelMat, dataMatrix, b, i)
            Ei = fXi - float(labelMat[i])
            if (labelMat[i]*Ei < -toler and alphas[i] < C) \
                or (labelMat[i]*Ei > toler and alphas[i] > 0):
                    j = selectJrand(i, m)
                    fXj = fX(alphas, labelMat, dataMatrix, b, j)
                    Ej = fXj - float(labelMat[j])
                    alphaIold = alphas[i].copy()
                    alphaJold = alphas[j].copy()
                    if labelMat[i] != labelMat[j]:
                        L = max(0, alphas[j] - alphas[i])
                        H = min(C, C + alphas[j] - alphas[i])
                    else:
                        L = max(0, alphas[j] + alphas[i] - C)
                        H = min(C, alphas[j] + alphas[i])
                    if L == H:
                        print 'L==H'
                        continue
                    eta = 2.0 * dataMatrix[i,:] * dataMatrix[j,:].T \
                        - dataMatrix[i,:] * dataMatrix[i,:].T \
                        - dataMatrix[j,:] * dataMatrix[j,:].T
                    if eta >= 0:
                        print 'eta >= 0'
                        continue
                    alphas[j] -= labelMat[j]*(Ei - Ej)/eta
                    alphas[j] = clipAlpha(alphas[j], H, L)
                    if abs(alphas[j] - alphaJold) < 0.0001:
                        print 'j not moving enough'
                        continue
                    # too sleepy 2017.7.4